In [1]:
import torch
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt 
from torchvision import datasets, transforms, models
from tqdm import tqdm

In [2]:
BATCH_SIZE = 32
EPOCHS = 5

#### Dataset Loading from Keras

In [3]:
from keras.datasets import cifar10
(xTr, yTr), (xTe, yTe) = keras.datasets.cifar10.load_data()

# consider 1 training batch as validation
xVal, yVal = xTr[:10000], yTr[:10000]
# consider 4 training batches
xTr, yTr = xTr[10000:], yTr[10000:]

#### Build fc6 Model

In [7]:
model = models.alexnet(pretrained=True)
# Get Classifiers
fc6 = torch.nn.Sequential(*list(model.classifier.children())[:2])
# change model
model.classifier = fc6
# send to GPU
model.cuda()
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

#### Training Step

In [8]:
def train( model, train_loader, optimizer):
    model.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()
    return loss.item(), correct / len(train_loader.dataset)

#### Testing/Evaluation Step

In [9]:
def test( model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.cuda(), target.cuda()
            output = model(data)
            test_loss += loss_fn(output, target).item() 
            pred = output.argmax(dim=1, keepdim=True)  
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    return test_loss,  correct / len(test_loader.dataset)

#### Loss Function

In [10]:
loss_fn = torch.nn.CrossEntropyLoss()

#### Optimizer

In [11]:
optimizer = torch.optim.Adam(model.parameters())

#### Preprocess

In [13]:
from PIL import Image
def preprocess(image):
    pil = Image.fromarray(image)  
    # change dimension to 224x224, convert to tensor, and normalize
    transform = transforms.Compose([
        transforms.Resize(256), 
        transforms.CenterCrop(224), 
        transforms.ToTensor(), 
        transforms.Normalize( 
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
        )
    ])
    img = transform(pil) 
    # add dimension
    return torch.unsqueeze(img, 0).cuda() if torch.cuda.is_available() else torch.unsqueeze(img, 0)

#### Running the Training

In [14]:
loss_history = []
acc_history = []
for i in tqdm(range(EPOCHS)):
    train_loss, train_acc = train(model, dataset, optimizer)
    val_loss, val_acc = test(model, val_dataset)
    loss_history.append((train_loss, val_loss))
    acc_history.append((train_acc, val_acc))

  0%|                                                                                                                                                                                                                | 0/50 [00:00<?, ?it/s]


RuntimeError: Expected 4-dimensional input for 4-dimensional weight [64, 3, 11, 11], but got 2-dimensional input of size [32, 3072] instead

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,5))
loss_history = np.array(loss_history)
acc_history = np.array(acc_history)
epochs_range = range(len(loss_history))

axs[0].plot(epochs_range, loss_history[:, 0])
axs[0].plot(epochs_range, loss_history[:, 1])
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('Loss')
axs[0].legend(['loss', 'val_loss'])

axs[1].plot(epochs_range, acc_history[:, 0])
axs[1].plot(epochs_range, acc_history[:, 1])
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('Accuracy')
axs[1].legend(['accuracy', 'val_accuracy'])

In [ ]:
loss, acc = test(model, test_dataset)
print(f"Test Dataset Loss: {loss}, Accuracy: {acc}")